# Regression Week 2: Multiple Regression (Interpretation)

In this notebook we will explore multiple regression and feature engineering with existing Turi Create functions.

We you will use data on house sales in King County to predict prices using multiple regression. We will:



* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

##To install turicreate or graphlab

In [ ]:
!pip install turicreate

In [ ]:
import turicreate as tc
from google.colab import files

In [ ]:
uploaded = files.upload()
!unzip home_data.sframe.zip

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
sales = tc.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [ ]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data.



In [ ]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = tc.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.010823     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame.

In [ ]:
example_weight_summary = example_model.coefficients
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019264  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions



In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] 

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. The function below is to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    predictions = model.predict(data)
    residuals = outcome - predictions
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print (rss_example_train) 

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)

In [ ]:
train_data[['bedrooms','bathrooms','lat','long','bedrooms_squared','bed_bath_rooms','log_sqft_living','lat_plus_long']].head()

bedrooms,bathrooms,lat,long,bedrooms_squared,bed_bath_rooms,log_sqft_living
3.0,1.0,47.51123398,-122.25677536,9.0,3.0,7.07326971746
3.0,2.25,47.72102274,-122.3188624,9.0,6.75,7.85166117789
2.0,1.0,47.73792661,-122.23319601,4.0,2.0,6.64639051485
4.0,3.0,47.52082,-122.39318505,16.0,12.0,7.58069975222
3.0,2.0,47.61681228,-122.04490059,9.0,6.0,7.4265490724
4.0,4.5,47.65611835,-122.00528655,16.0,18.0,8.59785109443
3.0,2.25,47.30972002,-122.32704857,9.0,6.75,7.4471683596
3.0,1.5,47.40949984,-122.31457273,9.0,4.5,6.96602418711
3.0,1.0,47.51229381,-122.33659507,9.0,3.0,7.48436864329
3.0,2.5,47.36840673,-122.0308176,9.0,7.5,7.54433210805


# Learning Multiple Models

We will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that we have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = tc.linear_regression.create(train_data, target='price',features=model_1_features, validation_set=None)
model_2 = tc.linear_regression.create(train_data, target='price',features=model_2_features, validation_set=None)
model_3 = tc.linear_regression.create(train_data, target='price',features=model_3_features, validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.042799     | 3193229.177888     | 228200.043155                   |

| 2         | 3        | 0.066577     | 3193229.177873     | 228200.043155                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [ ]:
# Examine/extract each model's coefficients:
model_1_weight_summary = model_1.coefficients
model_2_weight_summary = model_2.coefficients
model_3_weight_summary = model_3.coefficients
print (model_1_weight_summary) 
print (model_2_weight_summary)
print (model_3_weight_summary)

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56140675.7411 | 1649985.42023 |
| sqft_living |  None | 310.263325777  | 3.18882960407 |
|   bedrooms  |  None | -59577.1160676 | 2487.27977322 |
|  bathrooms  |  None | 13811.8405417  | 3593.54213297 |
|     lat     |  None | 629865.789471  | 13120.7100323 |
|     long    |  None | -214790.285165 | 13284.2851599 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54410676.1118 | 1650405.16537 |
|  sqft_living   |  None | 304.449298056  | 3.20217535637 |
|    bedrooms    |  None | -116366.043229 | 4805.54966548 |
|   bathrooms    |  None | -77972.330513  | 7565

What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1 and model 2? Why?



# Comparing multiple models



In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print rss_model_1_train
print rss_model_2_train
print rss_model_3_train

9.71328233545e+14
9.61592067858e+14
9.05276314552e+14


Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print rss_model_1_test
print rss_model_2_test
print rss_model_3_test

2.26568089093e+14
2.24368799994e+14
2.51829318963e+14
